In [2]:
import pandas as pd

csv_file_path = './raw_data/CWE2CVE Observed Examples - Página1.csv'  
cwe2cve = pd.read_csv(csv_file_path)

# Expanda a coluna "Observed Examples" para criar novas colunas
data = []

for index, row in cwe2cve.iterrows():
    observed_examples = row['Observed Examples'].split("::REFERENCE:")[1:]  # Dividir pelos CVEs
    for example in observed_examples:
        reference = example.split(":DESCRIPTION:")[0]
        description = example.split(":DESCRIPTION:")[1].split(":LINK:")[0]
        data.append({
            "REFERENCE": reference,
            "DESCRIPTION": description,
            "CWE-ID": row["CWE-ID"],
            "Name": row["Name"]
        })

# Criando o novo dataframe
df_expanded = pd.DataFrame(data)

df_expanded.rename(columns={
    "REFERENCE" : "cve_id",
    "DESCRIPTION": "cve_description",
    "CWE-ID": "cwe_id",
    "Name" : "cwe_description"
}, inplace=True)

# df_expanded.to_csv('cwe2cve_dataset.csv', index=False)

df_expanded.to_excel('output.xlsx', index=False)

print(df_expanded)

              cve_id                                    cve_description  \
0     CVE-2024-37032  Large language model (LLM) management tool doe...   
1     CVE-2022-45918  Chain: a learning management tool debugger use...   
2     CVE-2021-30860  Chain: improper input validation (CWE-20) lead...   
3     CVE-2021-30663  Chain: improper input validation (CWE-20) lead...   
4     CVE-2021-22205  Chain: backslash followed by a newline can byp...   
...              ...                                                ...   
2986  CVE-2021-33149  (RSB underflow, Retbleed). Shared return stack...   
2987   CVE-2024-3402  chain: GUI for ChatGPT API performs input vali...   
2988  CVE-2023-32786  Chain: LLM integration framework has prompt in...   
2989   CVE-2024-5184  ML-based email analysis product uses an API se...   
2990   CVE-2024-5565  Chain: library for generating SQL via LLMs usi...   

        cwe_id                                    cwe_description  
0       CWE-20                 

In [ ]:
import xml.etree.ElementTree as ET
import csv

# Substitua 'caminho_do_arquivo.xml' pelo caminho local do seu arquivo XML
xml_file = r"C:\Users\leo72\Documents\Mestrado\Disciplinas - Segundo Semestre de 2024\MC959A - TÓPICOS EM INTELIGÊNCIA ARTIFICIAL\Projeto de Conclusão\codigo\cwec_v4.16.xml"
output_file = 'cwe_dataset.csv'

# Parse o arquivo XML
tree = ET.parse(xml_file)
root = tree.getroot()

# Namespace para tratar as tags corretamente
namespace = {'ns': 'http://cwe.mitre.org/cwe-7'}

# Lista para armazenar os dados extraídos
data = []

# Função para limpar quebras de linha e espaços extras
def clean_text(text):
    return ' '.join(text.split()) if text else "No description available"

# Iterar pelas tags Weakness
for weakness in root.findall('.//ns:Weakness', namespace):
    cwe_id = f"CWE-{weakness.get('ID')}"  # Extrai o ID do CWE
    title = weakness.get('Name')  # Extrai o título (Name)
    description_elem = weakness.find('ns:Description', namespace)  # Extrai a descrição
    description_text = clean_text(description_elem.text) if description_elem is not None else "No description available"
    
    data.append({'cwe_id': cwe_id, 'title': title, 'description': description_text})

# Ordenar os dados por CWE-ID (inteiros)
data_sorted = sorted(data, key=lambda x: int(x['cwe_id'].split('-')[1]))

# Escrever os dados extraídos em um arquivo CSV
with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['cwe_id', 'title', 'description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(data_sorted)

print(f"Arquivo CSV gerado com sucesso: {output_file}")

Arquivo CSV gerado com sucesso: cwe_dataset.csv
